<a href="https://www.kaggle.com/code/nadaarfaoui/extracting-features-and-usage-of-a-product?scriptVersionId=289231569" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
import re

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

def generate_features(product_name: str, brand: str):
    prompt = f"""
List the top 3 features of the product "{product_name}" (brand: {brand})
and provide a short usage example. Return strictly in JSON with keys:
"feature1", "feature2", "feature3", "usage_example".
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract JSON
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except json.JSONDecodeError:
            pass

    # fallback
    return {"feature1": "", "feature2": "", "feature3": "", "usage_example": text}

# Test
report = generate_features("OnePlus Nord CE 2 Lite", "OnePlus")
print(report)
